# Sensor Validation Tutorial

This notebook guides you through validating the sensor simulation in the Digital Twin project, covering LiDAR, depth camera, and IMU sensors with their noise models.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import math
import time

print("Sensor Validation Tutorial")
print("=" * 30)
print("Validating LiDAR, Depth Camera, and IMU sensors with realistic noise models")

## 1. Sensor Specifications

Review the specifications for each sensor in our simulation.

In [ ]:
# Define sensor specifications
sensors = {
    "LiDAR": {
        "type": "Ray sensor",
        "fov": 180,  # degrees
        "samples": 720,
        "resolution": 0.5,  # degrees per sample
        "range": (0.1, 30.0),  # min, max in meters
        "update_rate": 10,  # Hz
        "noise_stddev": 0.01  # meters
    },
    "Depth Camera": {
        "type": "Depth sensor",
        "resolution": (640, 480),  # width, height
        "fov": 60,  # degrees
        "range": (0.1, 10.0),  # min, max in meters
        "update_rate": 30,  # Hz
        "noise_base": 0.007  # meters at 1m
    },
    "IMU": {
        "type": "Inertial measurement unit",
        "update_rate": 100,  # Hz
        "gyro_noise": 0.0017,  # rad/s stddev
        "accel_noise": 0.017,  # m/s² stddev
        "gyro_drift": 0.0001,  # rad/s per sample
        "accel_drift": 0.001   # m/s² per sample
    }
}

print("Sensor Specifications:")
for name, spec in sensors.items():
    print(f"\n{name}:")
    for key, value in spec.items():
        print(f"  {key}: {value}")

## 2. LiDAR Simulation and Validation

Simulate and validate LiDAR sensor data with noise models.

In [ ]:
# Simulate LiDAR data
def simulate_lidar_scan(num_samples=720, fov=180, objects=None, noise_stddev=0.01):
    """Simulate a LiDAR scan with realistic objects and noise"""
    if objects is None:
        # Default objects: wall in front, some obstacles
        objects = [
            {"angle": 0, "distance": 2.0, "size": 0.5},  # Front wall
            {"angle": 45, "distance": 1.5, "size": 0.3},  # Side obstacle
            {"angle": -45, "distance": 1.2, "size": 0.4}  # Other side obstacle
        ]
    
    angles = np.linspace(-fov/2, fov/2, num_samples)
    ranges = np.full(num_samples, 30.0)  # Default to max range
    
    # Add objects to scan
    for obj in objects:
        obj_angle_idx = int((obj["angle"] + fov/2) / fov * num_samples)
        if 0 <= obj_angle_idx < num_samples:
            # Add object points around the main angle
            angle_range = int(obj["size"] / 2 / (fov / num_samples) * num_samples) or 1
            start_idx = max(0, obj_angle_idx - angle_range)
            end_idx = min(num_samples, obj_angle_idx + angle_range)
            
            for i in range(start_idx, end_idx):
                if ranges[i] > obj["distance"]:
                    ranges[i] = obj["distance"]
    
    # Add noise
    noise = np.random.normal(0, noise_stddev, num_samples)
    noisy_ranges = ranges + noise
    
    # Ensure ranges are within bounds
    noisy_ranges = np.clip(noisy_ranges, 0.1, 30.0)
    
    return angles, noisy_ranges

# Generate a sample scan
angles, ranges = simulate_lidar_scan()

# Visualization
plt.figure(figsize=(12, 5))

# Plot 1: Range vs Angle
plt.subplot(1, 2, 1)
plt.plot(angles, ranges, 'b-', alpha=0.7)
plt.xlabel('Angle (degrees)')
plt.ylabel('Range (meters)')
plt.title('LiDAR Scan with Noise')
plt.grid(True, alpha=0.3)

# Plot 2: Polar plot
plt.subplot(1, 2, 2, projection='polar')
angles_rad = np.radians(angles)
plt.plot(angles_rad, ranges, 'b-', alpha=0.7)
plt.title('LiDAR Scan (Polar)')
plt.grid(True)

plt.tight_layout()
plt.show()

print(f"LiDAR scan generated: {len(ranges)} samples")
print(f"Range statistics - Min: {np.min(ranges):.2f}m, Max: {np.max(ranges):.2f}m, Mean: {np.mean(ranges):.2f}m")
print(f"Range standard deviation: {np.std(ranges):.3f}m")

## 3. Depth Camera Simulation and Validation

Simulate and validate depth camera data with realistic noise models.

In [ ]:
# Simulate depth camera data
def simulate_depth_image(width=640, height=480, fov=60, noise_factor=0.007):
    """Simulate a depth image with realistic noise characteristics"""
    
    # Create a synthetic scene: planar surface with some objects
    depth_map = np.zeros((height, width), dtype=np.float32)
    
    center_x, center_y = width / 2, height / 2
    fov_rad = math.radians(fov)
    
    for y in range(height):
        for x in range(width):
            # Calculate angle from center
            dx = (x - center_x) / center_x
            dy = (y - center_y) / center_y
            angle = math.sqrt(dx*dx + dy*dy) * fov_rad / 2
            
            # Simulate distance: closer in center, farther on edges
            distance = 1.0 + 4.0 * math.sqrt((x-center_x)**2 + (y-center_y)**2) / math.sqrt(center_x**2 + center_y**2)
            
            # Add distance-dependent noise
            noise = np.random.normal(0, noise_factor * distance)
            depth_map[y, x] = max(0.1, distance + noise)  # Ensure minimum distance
    
    return depth_map

# Generate a sample depth image
depth_img = simulate_depth_image()

# Visualization
plt.figure(figsize=(15, 5))

# Original depth map
plt.subplot(1, 3, 1)
plt.imshow(depth_img, cmap='viridis')
plt.colorbar(label='Depth (m)')
plt.title('Simulated Depth Image')

# Histogram of depth values
plt.subplot(1, 3, 2)
plt.hist(depth_img.flatten(), bins=100, range=(0, 10), alpha=0.7)
plt.xlabel('Depth (m)')
plt.ylabel('Frequency')
plt.title('Depth Value Distribution')

# Cross-section through center
plt.subplot(1, 3, 3)
center_row = depth_img[depth_img.shape[0]//2, :]
center_col = depth_img[:, depth_img.shape[1]//2]
plt.plot(center_row, label='Horizontal center', alpha=0.7)
plt.plot(center_col, label='Vertical center', alpha=0.7)
plt.xlabel('Pixel')
plt.ylabel('Depth (m)')
plt.title('Center Cross-Sections')
plt.legend()

plt.tight_layout()
plt.show()

print(f"Depth image generated: {depth_img.shape[1]}x{depth_img.shape[0]}")
print(f"Depth statistics - Min: {np.min(depth_img):.2f}m, Max: {np.max(depth_img):.2f}m, Mean: {np.mean(depth_img):.2f}m")
print(f"Depth standard deviation: {np.std(depth_img):.3f}m")

## 4. IMU Simulation and Validation

Simulate and validate IMU data with realistic noise and bias characteristics.

In [ ]:
# Simulate IMU data
def simulate_imu_data(duration=10, update_rate=100):
    """Simulate IMU data with realistic noise and bias drift"""
    
    dt = 1.0 / update_rate
    num_samples = int(duration * update_rate)
    
    # Parameters from our IMU model
    gyro_noise = 0.0017  # rad/s
    accel_noise = 0.017  # m/s^2
    gyro_drift_rate = 0.0001  # rad/s per sample
    accel_drift_rate = 0.001  # m/s^2 per sample
    
    # Initialize arrays
    timestamps = np.arange(0, duration, dt)
    
    # True values (robot mostly stationary)
    true_gyro = np.zeros((num_samples, 3))  # Mostly zero with small movements
    true_accel = np.zeros((num_samples, 3))  # Gravity vector with small movements
    
    # Add small oscillations to gyro
    t = np.arange(num_samples) * dt
    true_gyro[:, 0] = 0.01 * np.sin(0.5 * t)  # Small angular velocities
    true_gyro[:, 1] = 0.01 * np.cos(0.3 * t)
    true_gyro[:, 2] = 0.005 * np.sin(0.7 * t)
    
    # Acceleration: mostly gravity with small movements
    true_accel[:, 0] = 0.1 * np.sin(0.2 * t)
    true_accel[:, 1] = 0.1 * np.cos(0.3 * t)
    true_accel[:, 2] = 9.81  # Gravity
    
    # Simulate bias drift
    gyro_bias = np.zeros((num_samples, 3))
    accel_bias = np.zeros((num_samples, 3))
    
    # Accumulate bias drift over time
    for i in range(1, num_samples):
        gyro_bias[i] = gyro_bias[i-1] + np.random.normal(0, gyro_drift_rate, 3)
        accel_bias[i] = accel_bias[i-1] + np.random.normal(0, accel_drift_rate, 3)
    
    # Add noise to true values
    gyro_noise_samples = np.random.normal(0, gyro_noise, (num_samples, 3))
    accel_noise_samples = np.random.normal(0, accel_noise, (num_samples, 3))
    
    measured_gyro = true_gyro + gyro_noise_samples + gyro_bias
    measured_accel = true_accel + accel_noise_samples + accel_bias
    
    return timestamps[:len(true_gyro)], true_gyro, measured_gyro, true_accel, measured_accel, gyro_bias, accel_bias

# Generate sample IMU data
timestamps, true_gyro, meas_gyro, true_accel, meas_accel, gyro_bias, accel_bias = simulate_imu_data()

# Visualization
fig, axes = plt.subplots(3, 2, figsize=(15, 12))

# Gyroscope data
axes[0, 0].plot(timestamps, true_gyro[:, 0], label='True X', alpha=0.7)
axes[0, 0].plot(timestamps, meas_gyro[:, 0], label='Measured X', alpha=0.7)
axes[0, 0].set_xlabel('Time (s)')
axes[0, 0].set_ylabel('Angular Velocity (rad/s)')
axes[0, 0].set_title('Gyroscope X-axis')
axes[0, 0].legend()
axes[0, 0].grid(True, alpha=0.3)

axes[0, 1].plot(timestamps, true_gyro[:, 1], label='True Y', alpha=0.7)
axes[0, 1].plot(timestamps, meas_gyro[:, 1], label='Measured Y', alpha=0.7)
axes[0, 1].set_xlabel('Time (s)')
axes[0, 1].set_ylabel('Angular Velocity (rad/s)')
axes[0, 1].set_title('Gyroscope Y-axis')
axes[0, 1].legend()
axes[0, 1].grid(True, alpha=0.3)

# Accelerometer data
axes[1, 0].plot(timestamps, true_accel[:, 0], label='True X', alpha=0.7)
axes[1, 0].plot(timestamps, meas_accel[:, 0], label='Measured X', alpha=0.7)
axes[1, 0].set_xlabel('Time (s)')
axes[1, 0].set_ylabel('Linear Accel (m/s²)')
axes[1, 0].set_title('Accelerometer X-axis')
axes[1, 0].legend()
axes[1, 0].grid(True, alpha=0.3)

axes[1, 1].plot(timestamps, true_accel[:, 2], label='True Z', alpha=0.7)
axes[1, 1].plot(timestamps, meas_accel[:, 2], label='Measured Z', alpha=0.7)
axes[1, 1].set_xlabel('Time (s)')
axes[1, 1].set_ylabel('Linear Accel (m/s²)')
axes[1, 1].set_title('Accelerometer Z-axis (gravity)')
axes[1, 1].legend()
axes[1, 1].grid(True, alpha=0.3)

# Bias drift
axes[2, 0].plot(timestamps, gyro_bias[:, 0], label='X bias', alpha=0.7)
axes[2, 0].plot(timestamps, gyro_bias[:, 1], label='Y bias', alpha=0.7)
axes[2, 0].plot(timestamps, gyro_bias[:, 2], label='Z bias', alpha=0.7)
axes[2, 0].set_xlabel('Time (s)')
axes[2, 0].set_ylabel('Gyro Bias (rad/s)')
axes[2, 0].set_title('Gyroscope Bias Drift')
axes[2, 0].legend()
axes[2, 0].grid(True, alpha=0.3)

axes[2, 1].plot(timestamps, accel_bias[:, 2], label='Z bias', alpha=0.7)
axes[2, 1].set_xlabel('Time (s)')
axes[2, 1].set_ylabel('Accel Bias (m/s²)')
axes[2, 1].set_title('Accelerometer Bias Drift (Z-axis)')
axes[2, 1].legend()
axes[2, 1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print(f"IMU data generated: {len(timestamps)} samples over {timestamps[-1]:.1f} seconds")
print(f"Gyro magnitude - Mean: {np.mean(np.linalg.norm(meas_gyro, axis=1)):.4f} rad/s")
print(f"Accel magnitude - Mean: {np.mean(np.linalg.norm(meas_accel, axis=1)):.4f} m/s²")
print(f"Expected gravity magnitude: 9.81 m/s²")

## 5. Sensor Fusion Example

Combine data from multiple sensors to create a more complete picture of the environment.

In [ ]:
# Simulate sensor fusion
def create_point_cloud_from_lidar_and_imu(lidar_angles, lidar_ranges, imu_orientation):
    """Create a simple point cloud from LiDAR data, corrected with IMU orientation"""
    
    # Convert polar coordinates to Cartesian
    x = lidar_ranges * np.cos(np.radians(lidar_angles))
    y = lidar_ranges * np.sin(np.radians(lidar_angles))
    
    # For simplicity, assume all points are at z=0
    z = np.zeros_like(x)
    
    # Stack into points array
    points = np.column_stack([x, y, z])
    
    # Apply simple orientation correction (in a real system this would be more complex)
    # For now, just return the basic point cloud
    return points

# Create a sample point cloud
angles, ranges = simulate_lidar_scan(num_samples=360)  # Lower resolution for visualization

# For simplicity, using identity orientation
sample_orientation = np.array([0.0, 0.0, 0.0])  # [roll, pitch, yaw]

# Generate point cloud
points = create_point_cloud_from_lidar_and_imu(angles, ranges, sample_orientation)

# Visualization
fig = plt.figure(figsize=(12, 5))

# 2D projection
plt.subplot(1, 2, 1)
plt.scatter(points[:, 0], points[:, 1], s=1, alpha=0.6)
plt.xlabel('X (m)')
plt.ylabel('Y (m)')
plt.title('Point Cloud from LiDAR Scan')
plt.grid(True, alpha=0.3)
plt.axis('equal')

# 3D visualization (even though z is 0)
ax = fig.add_subplot(1, 2, 2, projection='3d')
ax.scatter(points[:, 0], points[:, 1], points[:, 2], s=1, alpha=0.6)
ax.set_xlabel('X')
ax.set_ylabel('Y')
ax.set_zlabel('Z')
ax.set_title('3D Point Cloud')

plt.tight_layout()
plt.show()

print(f"Point cloud generated: {len(points)} points")
print(f"Point cloud bounds - X: ({np.min(points[:, 0]):.2f}, {np.max(points[:, 0]):.2f})")
print(f"Point cloud bounds - Y: ({np.min(points[:, 1]):.2f}, {np.max(points[:, 1]):.2f})")

## 6. Validation Summary

Validate that our simulated sensors meet the requirements for the Digital Twin project.

In [ ]:
# Validation checks
print("SENSOR VALIDATION SUMMARY")
print("=" * 50)

# LiDAR validation
lidar_valid = True
if np.min(ranges) >= 0.1 and np.max(ranges) <= 30.0:
    print("✅ LiDAR: Range values within expected bounds (0.1m - 30m)")
else:
    print("❌ LiDAR: Range values outside expected bounds")
    lidar_valid = False

# Depth camera validation
camera_valid = True
if depth_img.shape == (480, 640) and np.min(depth_img) >= 0.1 and np.max(depth_img) <= 10.0:
    print("✅ Depth Camera: Resolution and range values within expected bounds")
else:
    print("❌ Depth Camera: Resolution or range values outside expected bounds")
    camera_valid = False

# IMU validation
imu_valid = True
avg_gravity = np.mean(meas_accel[:, 2])
if 8.0 < avg_gravity < 12.0:  # Gravity should be around 9.81
    print(f"✅ IMU: Average Z-acceleration {avg_gravity:.2f} m/s² (within expected range)")
else:
    print(f"❌ IMU: Average Z-acceleration {avg_gravity:.2f} m/s² (unexpected)")
    imu_valid = False

# Check noise levels
range_std = np.std(ranges)
if range_std > 0.001:  # Ensure some noise is present
    print(f"✅ LiDAR: Appropriate noise level present (stddev: {range_std:.3f}m)")
else:
    print(f"⚠️  LiDAR: Low noise level (stddev: {range_std:.3f}m)")
    lidar_valid = False

print(f"\nValidation Results:")
print(f"  LiDAR: {'PASS' if lidar_valid else 'FAIL'}")
print(f"  Depth Camera: {'PASS' if camera_valid else 'FAIL'}")
print(f"  IMU: {'PASS' if imu_valid else 'FAIL'}")

overall_valid = lidar_valid and camera_valid and imu_valid
print(f"  Overall: {'PASS' if overall_valid else 'FAIL'}")

if overall_valid:
    print("\n🎉 All sensors successfully validated with realistic characteristics!")
else:
    print("\n⚠️  Some sensors did not meet validation criteria. Check configuration.")

# Print detailed statistics
print(f"\nDETAILED STATISTICS:")
print(f"  LiDAR - Samples: {len(ranges)}, Min: {np.min(ranges):.2f}m, Max: {np.max(ranges):.2f}m")
print(f"  Depth Camera - Resolution: {depth_img.shape[1]}x{depth_img.shape[0]}, Mean depth: {np.mean(depth_img):.2f}m")
print(f"  IMU - Avg gyro: [{np.mean(meas_gyro[:, 0]):.4f}, {np.mean(meas_gyro[:, 1]):.4f}, {np.mean(meas_gyro[:, 2]):.4f}] rad/s")
print(f"        Avg accel: [{np.mean(meas_accel[:, 0]):.4f}, {np.mean(meas_accel[:, 1]):.4f}, {np.mean(meas_accel[:, 2]):.4f}] m/s²")